# Acquire & Summarize

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from acquire import get_connection, get_zillow_data

import warnings
warnings.filterwarnings("ignore")

## 1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. 
- Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.
- Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
- Only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property ID's), along with zestimate error and date of transaction.
- Only include properties that include a latitude and longitude value.

In [ ]:
df = get_zillow_data()

## 2. Summarize your data 
- (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df = df.drop(df.columns[[59]], axis = 1)

<div class="alert alert-block alert-info">
Takeaways:
    
- Dropping duplicated id columns
    
</div>

In [ ]:
df.describe().T

In [ ]:
for col in df.columns:
    if df[col].dtype != object:
        plt.figure(figsize=(4,3))
        plt.hist(df[col])
        plt.title(col)
        plt.show()

In [ ]:
for col in df.columns:
    print("\n")
    print(df[col].value_counts())
    print("\n")

## 3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where 
- each row is an atttribute name
- the first column is the number of rows with missing values for that attribute
- the second column is percent of total rows that have missing values for that attribute. 
- Run the function and document takeaways from this on how you want to handle missing values.

In [ ]:
def null_finder_columns(df):
    '''
    This function takes in a DataFrame and list 
    information about the null values in the columns
    '''
    #accepts a 'df' and creates a new one labeled 'nulls'  
    #nulls index is the df's columns
    nulls = pd.DataFrame(index = df.columns)
    #sums up the null values in the dataframes columns
    nulls['num_rows_missing'] = df.isnull().sum(axis = 0)
    #finds the percentage of null values in the df's columns
    nulls['pct_rows_missing'] = nulls.num_rows_missing / df.shape[0]
    return nulls

In [ ]:
null_columns = null_finder_columns(df)

In [ ]:
null_columns.sort_values(by = 'pct_rows_missing', ascending = False).head(30)



<div class="alert alert-block alert-info">
Takeaways: 
    
- There are 34 out of the 68 columns missing more than half the value
    
- May need to drop a lot of the data  
    
</div>

## 4. Write a function that takes in a dataframe and returns a dataframe with 3 columns: 
- the number of columns missing, 
- percent of columns missing, 
- number of rows with n columns missing. 
- Run the function and document takeaways from this on how you want to handle missing values.

In [ ]:
def null_finder_rows(df):
    '''
    This function finds the number of columns missing in a row,
    the percent of columns missing in the row
    and the number of rows that have the same amount of columns missing
    '''
    #initiate a dataframe
    rows = pd.DataFrame()
    #find the number of columns missing in the row
    rows['num_cols_missing'] = df.isnull().sum(axis=1)
    #find the percentage of columns missing in the row
    rows['pct_cols_missing'] = df.isnull().sum(axis=1) / df.shape[1]
    #group by 'num_cols_missing' and find 
    #how many rows have that number of columns missing
    num_rows = rows.groupby('num_cols_missing').count()
    #rename the column as 'num_rows'
    num_rows = num_rows.rename(columns ={'pct_cols_missing': "num_rows"})
    #group by 'num_cols_missing' and find 
    #the percentage of columns missing in the row
    pct_cols = rows.groupby('num_cols_missing').mean()
    #combine the 'pct_cols' and 'num_rows'
    result = pd.concat([pct_cols, num_rows], axis=1, sort=False)
    #take the 'num_cols_missing' out of the index
    result = result.reset_index()
    return result

In [ ]:
null_rows = null_finder_rows(df)

In [ ]:
null_rows.sort_values(by = 'pct_cols_missing', ascending = False).head(10)

<div class="alert alert-block alert-info">
Takeaways:

- Below 60% is a major jump in rows missing

- This will be the threshold 
    
</div>

# Prepare

## 1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...).
- There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. 
- Add some new logic that will reduce the number of properties that are falsely removed. 
- You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

In [ ]:
df.propertylandusedesc.value_counts()


<div class="alert alert-block alert-info">
Takeaways:

- keep only single family residential(261), residential general(260) 
    
</div>

In [ ]:
df = df[df.propertylandusetypeid.isin([260,261])]

In [ ]:
df.shape

- Check bedroom and bathroom counts

In [ ]:
df.bedroomcnt.value_counts()

In [ ]:
df.bathroomcnt.value_counts()

<div class="alert alert-block alert-info">
Takeaways:

- 178 houses had either 0 as bedroom or bathroom
    
</div>

In [ ]:
df = df[(df.bedroomcnt > 0) & (df.bathroomcnt > 0)]

In [ ]:
df.shape

In [ ]:
df.unitcnt.value_counts(dropna=False)


<div class="alert alert-block alert-info">
Takeaways:

- 18541 null unit counts. They are probably unit counts of 1.

- Filter out the units that are more than one

    
</div>

In [ ]:
df.unitcnt = df.unitcnt.fillna(1.0)

In [ ]:
df = df[df.unitcnt == 1]

In [ ]:
(df.unitcnt != 1).sum()

In [ ]:
df.shape

## 2. Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

- The input:
    - A dataframe
    - A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
    - A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
- The output:
    - The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
- hint:
    - Look up the dropna documentation.
    - You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
    - Make use of inplace, i.e. inplace=True/False.

In [ ]:
def handle_missing_values(df, prop_required_column, prop_required_row):
    thresh_col = int(round(prop_required_column*df.shape[0],0))
    df.dropna(axis=1, thresh=thresh_col, inplace=True)
    thresh_row = int(round(prop_required_row*df.shape[1],0))
    df.dropna(axis=0, thresh=thresh_row, inplace=True)
    return df 

In [ ]:
df = handle_missing_values(df, .6, .6)

## 3. Decide how to handle the remaining missing values:

   - Fill with constant value.
   - Impute with mean, median, mode.
   - Drop row/column

In [ ]:
df.info()

<div class="alert alert-block alert-info">
Takeaways:

- dropping typeid that are no longer needed
    
</div>

In [ ]:
df = df.drop(columns=["id", "propertylandusetypeid", "heatingorsystemtypeid", "unitcnt", "propertylandusedesc", "propertycountylandusecode"])

In [ ]:
df.isnull().sum().sort_values(ascending = False)

<div class="alert alert-block alert-info">
Takeaways:

- propertyzoningdesc = Description of the allowed land uses (zoning) for that property
- dropping because we already filtered for single unit residential.
- let's take a closer look into the missing values
    
</div>

In [ ]:
df = df.drop(columns=["propertyzoningdesc"])

In [ ]:
df.heatingorsystemdesc.value_counts(dropna = False)

<div class="alert alert-block alert-info">
Takeaways:

- In southern California, the NaN values are most likely no type of heating system

- We will fill Nan with none
    
</div>

In [ ]:
df.heatingorsystemdesc = df.heatingorsystemdesc.fillna("None")

In [ ]:
(df.calculatedbathnbr == df.bathroomcnt).sum() / len(df)

<div class="alert alert-block alert-info">
Takeaways:

- Looks like calculatedbathrbr is equivalent to bathroomcnt
- drop the duplicate column
    
</div>

In [ ]:
df = df.drop(columns=["calculatedbathnbr"])

<div class="alert alert-block alert-info">
Takeaways:

- We have dropped all we can
- To fill the rest of the NaNs we should split the data
    
</div>

In [ ]:
(df.finishedsquarefeet12 == df.calculatedfinishedsquarefeet).sum() / len(df)

<div class="alert alert-block alert-info">
Takeaways:

- Looks like finishedsquarefeet12 is equilalent to calculatedfinishedsqurefeet
- drop the duplicate column
    
</div>

In [ ]:
df = df.drop(columns = ['finishedsquarefeet12'])

In [ ]:
df['county'] = df.fips.replace([6037, 6059, 6111],['los_angeles', 'orange', 'ventura'])

In [ ]:
df = df.drop(columns = ['fips'])

<div class="alert alert-block alert-info">
Takeaways:

- change fips number to a category labeled by county
- drop the duplicate column
    
</div>

## Train, Validate, and Test Splits

In [ ]:
from sklearn.model_selection import train_test_split

train_and_validate, test = train_test_split(df, train_size=.8, random_state=123)
train, validate = train_test_split(train_and_validate, train_size = .7, random_state=123)

In [ ]:
train.shape, validate.shape, test.shape

In [ ]:
df.isnull().sum().sort_values(ascending = False).head(15)

<div class="alert alert-block alert-info">
Takeaways:

- We will divide the remaining nulls into two columns
    - The fixed values will use the mode
    - the continuous values will use the median
    
</div>

In [ ]:
cols_fixed = [
    'buildingqualitytypeid',
    'regionidcity',
    'censustractandblock',
    'regionidzip',
    'yearbuilt'
]


for col in cols_fixed:
    mode = int(train[col].mode())
    train[col].fillna(value = mode, inplace = True)
    validate[col].fillna(value = mode, inplace = True)
    test[col].fillna(value = mode, inplace = True)

In [ ]:
train.isnull().sum().sort_values(ascending = False).head(15)

In [ ]:
cols_cont = [
    'lotsizesquarefeet',
    'structuretaxvaluedollarcnt',
    'fullbathcnt',
    'calculatedfinishedsquarefeet',
    'taxamount',
    'landtaxvaluedollarcnt',
    'taxvaluedollarcnt'
]


for col in cols_cont:
    median = train[col].median()
    train[col].fillna(median, inplace=True)
    validate[col].fillna(median, inplace=True)
    test[col].fillna(median, inplace=True)

In [ ]:
train.isnull().sum().sort_values(ascending=False)

In [ ]:
validate.isnull().sum().sort_values(ascending=False)

In [ ]:
test.isnull().sum().sort_values(ascending=False)

- Since there are no more null values and the data is prepped, we can write the output to disk

train.to_csv("zillow_train.csv")
validate.to_csv("zillow_validate.csv")
test.to_csv("zillow_test.csv")